<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Content-Based Personalization with LightGBM on Spark

This notebook provides a quick example of how to train a [LightGBM](https://github.com/Microsoft/Lightgbm) model on Spark using [MMLSpark](https://github.com/Azure/mmlspark) for a content-based personalization scenario.

We use the [CRITEO dataset](https://www.kaggle.com/c/criteo-display-ad-challenge), a well known dataset of website ads that can be used to optimize the Click-Through Rate (CTR). The dataset consists of a series of numerical and categorical features and a binary label indicating whether the add has been clicked or not.

The model is based on [LightGBM](https://github.com/Microsoft/Lightgbm), which is a gradient boosting framework that uses tree-based learning algorithms. Finally, we take advantage of
[MMLSpark](https://github.com/Azure/mmlspark) library, which allows LightGBM to be called in a Spark environment and be computed distributely.

This scenario is a good example of **implicit feedback**, where binary labels indicate the interaction between a user and an item. This contrasts with explicit feedback, where the user explicitely rate the content, for example from 1 to 5. 


## Global Settings and Imports

This notebook can be run in a Spark environment in a DSVM or in Azure Databricks. For more details about the installation process, please refer to the [setup instructions](../../SETUP.md).

**NOTE for Azure Databricks:**
* A python script is provided to simplify setting up Azure Databricks with the correct dependencies. Run ```python tools/databricks_install.py -h``` for more details.
* MMLSpark should not be run on a cluster with autoscaling enabled. Disable the flag in the Azure Databricks Cluster configuration before running this notebook.

In [14]:
!python C:/Users/dagar/Documents/Github/recommenders/tools/databricks_install.py -h


usage: databricks_install.py [-h] [--profile PROFILE]
                             [--path-to-recommenders PATH_TO_RECOMMENDERS]
                             [--eggname EGGNAME] [--dbfs-path DBFS_PATH]
                             [--overwrite] [--prepare-o16n] [--mmlspark]
                             [--create-cluster]
                             cluster_id

This script packages the recommenders directory into a .egg file and installs
it onto a databricks cluster. Optionally, this script may also install the
mmlspark library, and it may also install additional libraries useful for
operationalization. This script requires that you have installed databricks-
cli in the python environment in which you are running this script, and that
have you have already configured it with a profile.

positional arguments:
  cluster_id            cluster id for the cluster to install data on. If used
                        in conjunction with --create-cluster, this is the name
                      

In [15]:
import os
import sys


import pyspark
from pyspark.ml import PipelineModel
from pyspark.ml.feature import FeatureHasher
import papermill as pm
import scrapbook as sb

from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.utils.notebook_utils import is_databricks
from recommenders.datasets.criteo import load_spark_df
from recommenders.datasets.spark_splitters import spark_random_split

# Setup MML Spark
if not is_databricks():
    # get the maven coordinates for MML Spark from databricks_install script
    from databricks_install import MMLSPARK_INFO
    packages = [MMLSPARK_INFO["maven"]["coordinates"]]
    repo = MMLSPARK_INFO["maven"].get("repo")
    #spark = start_or_get_spark(packages="Azure:mmlspark:0.16")
    spark = start_or_get_spark(packages=packages, repository=repo)
    dbutils = None
    print("MMLSpark version: {}".format(MMLSPARK_INFO['maven']['coordinates']))

from mmlspark.train import ComputeModelStatistics
from mmlspark.lightgbm import LightGBMClassifier

print("System version: {}".format(sys.version))
print("PySpark version: {}".format(pyspark.version.__version__))

MMLSpark version: com.microsoft.ml.spark:mmlspark_2.11:0.18.1
System version: 3.6.11 (default, Nov 27 2020, 18:37:51) [MSC v.1916 64 bit (AMD64)]
PySpark version: 2.4.3


In [37]:
# Criteo data size, it can be "sample" or "full"
DATA_SIZE = "sample"

# LightGBM parameters
# More details on parameters: https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
NUM_LEAVES = 12
NUM_ITERATIONS = 50
LEARNING_RATE = 0.1
FEATURE_FRACTION = 0.8
EARLY_STOPPING_ROUND = 10

# Model name
MODEL_PATH = '/lightgbm_criteo.mml'

## Data Preparation

The [Criteo Display Advertising Challenge](https://www.kaggle.com/c/criteo-display-ad-challenge) (Criteo DAC) dataset is a well-known industry benchmarking dataset for developing CTR prediction models, and is used frequently by research papers. The original dataset contains over 45M rows, but there is also a down-sampled dataset which has 100,000 rows (this can be used by setting `DATA_SIZE = "sample"`). Each row corresponds to a display ad served by Criteo and the first column is indicates whether this ad has been clicked or not.<br><br>
The dataset contains 1 label column and 39 feature columns, where 13 columns are integer values (int00-int12) and 26 columns are categorical features (cat00-cat25).<br><br>
What the columns represent is not provided, but for this case we can consider the integer and categorical values as features representing the user and / or item content. The label is binary and is an example of implicit feedback indicating a user's interaction with an item. With this dataset we can demonstrate how to build a model that predicts the probability of a user interacting with an item based on available user and item content features.


In [17]:
raw_data = load_spark_df(size=DATA_SIZE, spark=spark, dbutils=dbutils)
# visualize data
raw_data.limit(2).toPandas().head()

100%|██████████| 8.58k/8.58k [00:23<00:00, 363KB/s]


,label,int00,int01,int02,int03,int04,int05,int06,int07,int08,...,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25
0,0,1,1,5,0,1382,4,15,2,181,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,None,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2,0,44,1,102,8,2,2,4,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,None,3a171ecb,43f13e8b,e8b83407,731c3655


In [18]:
raw_data.toPandas().shape

(100000, 40)

### Feature Processing
The feature data provided has many missing values across both integer and categorical feature fields. In addition the categorical features have many distinct values, so effectively cleaning and representing the feature data is an important step prior to training a model.<br><br>
One of the simplest ways of managing both features that have missing values as well as high cardinality is to use the hashing trick. The [FeatureHasher](http://spark.apache.org/docs/latest/ml-features.html#featurehasher) transformer will pass integer values through and will hash categorical features into a sparse vector of lower dimensionality, which can be used effectively by LightGBM.<br><br>
First, the dataset is splitted randomly for training and testing and feature processing is applied to each dataset.

In [19]:
raw_train, raw_test = spark_random_split(raw_data, ratio=0.8, seed=42)

In [20]:
columns = [c for c in raw_data.columns if c != 'label']
feature_processor = FeatureHasher(inputCols=columns, outputCol='features')

In [21]:
train = feature_processor.transform(raw_train)
test = feature_processor.transform(raw_test)

In [22]:
train.limit(2).head()

Row(label=0, int00=None, int01=-2, int02=None, int03=11, int04=589318, int05=None, int06=0, int07=18, int08=3, int09=None, int10=0, int11=None, int12=13, cat00='05db9164', cat01='d833535f', cat02='b00d1501', cat03='d16679b9', cat04='25c83c98', cat05='7e0ccccf', cat06='863329da', cat07='062b5529', cat08='7cc72ec2', cat09='d47daa4c', cat10='a89c45cb', cat11='e0d76380', cat12='a4fafa5b', cat13='07d13a8f', cat14='943169c2', cat15='1203a270', cat16='d4bb7bd8', cat17='281769c2', cat18=None, cat19=None, cat20='73d06dde', cat21='c9d4222a', cat22='c7dc6720', cat23='aee52b6f', cat24=None, cat25=None, features=SparseVector(262144, {1562: 1.0, 9567: 1.0, 20842: 1.0, 25911: 1.0, 31484: 1.0, 38328: 1.0, 46747: 18.0, 51885: 1.0, 54551: 1.0, 55096: 1.0, 61856: 1.0, 66551: 0.0, 68372: 1.0, 84264: 3.0, 92291: 1.0, 110783: 1.0, 127003: 589318.0, 131569: 1.0, 134624: 1.0, 170681: 1.0, 202012: 1.0, 212186: 1.0, 226453: 11.0, 238710: 1.0, 243754: 1.0, 248706: 1.0, 248972: 13.0, 256378: 1.0, 258299: 0.0, 261

In [23]:
train.schema

StructType(List(StructField(label,IntegerType,true),StructField(int00,IntegerType,true),StructField(int01,IntegerType,true),StructField(int02,IntegerType,true),StructField(int03,IntegerType,true),StructField(int04,IntegerType,true),StructField(int05,IntegerType,true),StructField(int06,IntegerType,true),StructField(int07,IntegerType,true),StructField(int08,IntegerType,true),StructField(int09,IntegerType,true),StructField(int10,IntegerType,true),StructField(int11,IntegerType,true),StructField(int12,IntegerType,true),StructField(cat00,StringType,true),StructField(cat01,StringType,true),StructField(cat02,StringType,true),StructField(cat03,StringType,true),StructField(cat04,StringType,true),StructField(cat05,StringType,true),StructField(cat06,StringType,true),StructField(cat07,StringType,true),StructField(cat08,StringType,true),StructField(cat09,StringType,true),StructField(cat10,StringType,true),StructField(cat11,StringType,true),StructField(cat12,StringType,true),StructField(cat13,StringT

## Model Training
In MMLSpark, the LightGBM implementation for binary classification is invoked using the `LightGBMClassifier` class and specifying the objective as `"binary"`. In this instance, the occurrence of positive labels is quite low, so setting the `isUnbalance` flag to true helps account for this imbalance.<br><br>

### Hyper-parameters
Below are some of the key [hyper-parameters](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst) for training a LightGBM classifier on Spark:
- `numLeaves`: the number of leaves in each tree
- `numIterations`: the number of iterations to apply boosting
- `learningRate`: the learning rate for training across trees
- `featureFraction`: the fraction of features used for training a tree
- `earlyStoppingRound`: round at which early stopping can be applied to avoid overfitting

In [24]:
lgbm = LightGBMClassifier(
    labelCol="label",
    featuresCol="features",
    objective="binary",
    isUnbalance=True,
    boostingType="gbdt",
    boostFromAverage=True,
    baggingSeed=42,
    numLeaves=NUM_LEAVES,
    numIterations=NUM_ITERATIONS,
    learningRate=LEARNING_RATE,
    featureFraction=FEATURE_FRACTION,
    earlyStoppingRound=EARLY_STOPPING_ROUND
)

### Model Training and Evaluation

In [25]:
model = lgbm.fit(train)


In [26]:
predictions = model.transform(test)

In [27]:
evaluator = (
    ComputeModelStatistics()
    .setScoredLabelsCol("prediction")
    .setLabelCol("label")
    .setEvaluationMetric("AUC")
)

result = evaluator.transform(predictions)
auc = result.select("AUC").collect()[0][0]
result.show()

+---------------+------------------+
|evaluation_type|               AUC|
+---------------+------------------+
| Classification|0.6828644257080493|
+---------------+------------------+



In [28]:
# Record results with papermill for tests
sb.glue("auc", auc)

## Model Saving 
The full pipeline for operating on raw data including feature processing and model prediction can be saved and reloaded for use in another workflow.

In [30]:
from pyspark.ml import Pipeline

In [31]:
# create pipeline
pipeline = Pipeline(stages=[feature_processor, model])


In [44]:
# save model
pipeline.write().overwrite().save('/tmp/lightgbm_criteo.mml')

Py4JJavaError: An error occurred while calling o868.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$$

In [45]:
# cleanup spark instance
if not is_databricks():
    spark.stop()

## Additional Reading
\[1\] Guolin Ke, Qi Meng, Thomas Finley, Taifeng Wang, Wei Chen, Weidong Ma, Qiwei Ye, and Tie-Yan Liu. 2017. LightGBM: A highly efficient gradient boosting decision tree. In Advances in Neural Information Processing Systems. 3146–3154. https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf <br>
\[2\] MML Spark: https://mmlspark.blob.core.windows.net/website/index.html <br>
